In [1]:
import tools.processing as pre
import tools.architectures as nn

In [2]:
text = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")

In [3]:
text[:100]

"as real as it seems the american dream\nain't nothing but another calculated schemes\nto get us locked"

In [5]:
alphabet = pre.Alphabet(text)

HIDDEN_LAYER_SIZE = 128
VOCAB_SIZE = alphabet.get_size()
TIMESTEPS = 50

EPOCHS = 20
BATCH_SIZE = 128


data, labels = alphabet.making_one_hot(text, TIMESTEPS)


rnn = nn.RNN(name = "basic")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0)

rnn.train(data, labels, alphabet, epochs=EPOCHS, batch_size=BATCH_SIZE)



Epoch 1/20
Loss:    	 3.4338860511779785
Accuracy:	 1.3829939365386963
Seed:as real as it seems the americ
Result:as real as it seems the americetuntoe t ew tt  s  a  o   yyet yay  neo  o ts  oe e o ioy  h  rny     a on e  oee    e ut toe aaed  h hivwaa  heir  am  r  s  ee se i in     tt rotot   ncs het oeite o e  toeoih i    yoo  t i eoea n  y  l et e eete     o    t e d a e e a
ou   rey t  t aioe entoe  w o yecsc n e aos  reyoy u tt  y e e   l  e ai omn     t ow    e te nt   hw hm  oa  en o o   t  r et  etrehs t   o rmto
  owy si o e oe  et  r oaeo   e le atr e  e deo  o  ra a   hl o t  t ate           et   e  i r      tt ns  irk h   


Epoch 2/20
Loss:    	 2.9471890926361084
Accuracy:	 17.840621948242188
Seed:as real as it seems the americ
Result:as real as it seems the americn ooio a
o ibr    oeea a   aa  et s o ton oa te  o e o e o  td  iw  ne ieto nayoc      h     e
 liut y r oy ao ee  k e e ant oor et g a  o t le    r yt el  ea oea o  m e'a i  r  e h tw   t t n  o  sh at e ai